# Initialize Notebook

In [2]:
from IPython.display import HTML

import warnings
import json
import ast

from mstr_robotics.navigation import answer_prompts
from openai import OpenAI
from mstr_robotics.mstr_classes import mstr_global
from mstr_robotics._helper import msic
from mstr_robotics._connectors import mstr_api
from mstr_robotics.report import rep, prompts, cube

from mstrio.connection import Connection

from mstr_robotics.mstr_pandas import df_helper
from mstr_robotics.user_RAG import keyword_processor, vectorDB_faisst,mstr_openAI,chat_bot

## Config

In [3]:
user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)

jupyter_path="..\\config\\jupyter_objects_d.json"
with open(jupyter_path, 'r') as file:
    jupyter_objects_d = json.load(file)

conn_params =  user_d["conn_params"]
sKey=user_d["sKey"]

#GUID of report wizzard & prompts
project_id=jupyter_objects_d["turtorial_RAG"]["project_id"]          #Tutorial project
#Mstr prompted report
rep_dos_id=jupyter_objects_d["chat_bot_rep"]["rep_dos_id"]
#GUID of output folder for generated reports
ai_rep_folder_id = jupyter_objects_d["chat_bot_rep"]["ai_rep_folder_id"]

#cubes generated in load_rag_cubes
cube_RAG_form_val_ans_name=jupyter_objects_d["turtorial_RAG"]["cube_RAG_form_val_ans_name"]  
cube_RAG_form_val_ans_id=jupyter_objects_d["turtorial_RAG"]["cube_RAG_form_val_ans_id"]  

cube_obj_prp_rel_name=jupyter_objects_d["turtorial_RAG"]["cube_obj_prp_rel_name"]  
cube_obj_prp_rel_id=jupyter_objects_d["turtorial_RAG"]["cube_obj_prp_rel_id"]  

cube_att_form_def_name=jupyter_objects_d["turtorial_RAG"]["cube_att_form_def_name"]  
cube_att_form_def_id=jupyter_objects_d["turtorial_RAG"]["cube_att_form_def_id"]


warnings.filterwarnings("ignore")
i_mstr_global = mstr_global()


i_rep = rep()
i_cube = cube()
i_df_helper = df_helper()
i_mstr_global = mstr_global()
i_msic = msic()
i_prompts = prompts()
i_mstr_api = mstr_api()
i_chat_bot_RAG=chat_bot()


i_keyword_processor=keyword_processor()
#i_vectorDB_faisst=vectorDB_faisst(sKey=sKey)
#i_openAI=OpenAI(sKey=sKey)
i_OpenAI=OpenAI
#i_mstr_openAI=mstr_openAI(sKey=sKey)
i_mstr_openAI=mstr_openAI()
#i_openai.api_key = sKey

#Open connection to MSTR
conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"

#conn1 = Connection(base_url=base_url,username=username,
#                  password=password,project_id=project_id)
#conn1.headers['Content-type'] = "application/json"

Connection to MicroStrategy Intelligence Server has been established.


## Load cubes 

In [5]:
#cube with key cols for vector DB
cube_l=[cube_RAG_form_val_ans_id,cube_obj_prp_rel_id]

#all mtdi cubes 
mtdi_cube_l=[cube_RAG_form_val_ans_id,cube_obj_prp_rel_id,cube_att_form_def_id]
mstr_rag_col_d=i_cube.get_mtdi_cube_col_id(conn,cube_l=mtdi_cube_l)
i_run_chat_request=answer_prompts(cube_obj_prp_rel_id=cube_obj_prp_rel_id,
                                  cube_RAG_form_val_ans_id=cube_RAG_form_val_ans_id,
                                  cube_att_form_def_id=cube_att_form_def_id,
                                  rep_dos_id=rep_dos_id)

## Load vector DB and key word processor

In [6]:
# here we load attribute form values and attribute and metric names
#we only load columns with the name key
key_l=[]
for cube_id in cube_l:
    key_df=i_cube.quick_query_cube(conn=conn,cube_id=cube_id)
    key_l.extend(key_df["key"].astype(str).to_list())

#vector_store=i_vectorDB_faisst.load_vector_store(key_l)
#i_vectorDB_faisst.load_vector_store(key_l)
i_keyword_processor.load_keyword_processor(key_l)

OlapCube object named: 'attribute_form_elements' with ID: 'B0A4AFCB4F8B53B4EB677D89B7A85CF0'


Downloading: 100%|██████████| 9/9 [02:01<00:00, 13.49s/it, rows=1072522]


OlapCube object named: 'obj_prp_rel' with ID: '1F0C564A40101C08ABF602AAB81B2324'


Downloading: 100%|██████████| 2/2 [00:03<00:00,  1.68s/it, rows=25479]


## ChatGPT query

In [7]:
temperature=0
max_tokens=1000
top_p=1
model="gpt-4o-mini"
#model="gpt-4o"

msg_t="Please show me the Revenue, Cost and Profit for the attributes Category, Call Center"
msg_t= msg_t + "and filter for the yaer 2021, 2022 and Call Center Atlanta and Revenue is greater than 200"
#msg_t= msg_t + ", for the top  5 customers by revenue"

#msg_t="Top 5 items by revenue in 2022"
#msg_t= msg_t + " and where the region name begins with S"
#msg_t="Please show me the Revenue, cost and profit for Berlin at level of the Category, Call Center and Employee"
key_word_l=i_keyword_processor.extract_keywords(msg_t=msg_t)
rag_rep_prp_l=[]

#rag_rep_prp_filt_d=get_rag_rep_prp_d(conn=conn,cube_id=cube_RAG_form_val_ans_id, mstr_rag_col_d=mstr_rag_col_d,rep_dos_id=rep_dos_id,key_word_l=key_word_l)


messages=i_chat_bot_RAG.split_AI_msg(msg_t,key_word_l)
json_t=i_mstr_openAI.call_open_AI(sKey=sKey,messages=messages, max_tokens=max_tokens,temperature=temperature, model=model)
json_t_d=json.loads(json_t["choices"][0]["message"]["content"])
msg_filter_t=json_t_d["filter"]
messages=i_chat_bot_RAG.filter_RAG_l(msg_t=msg_t,key_word_l=key_word_l,msg_filter_t=msg_filter_t)
json_f=i_mstr_openAI.call_open_AI(sKey=sKey,messages=messages, max_tokens=max_tokens,temperature=temperature, model=model)
json_f=json_f["choices"][0]["message"]["content"]
filter_d=ast.literal_eval(json_f)
rep_def_j={"template":json_t_d["template"],"filter":filter_d}
rep_def_j

{'template': {'metrics': ['Revenue', 'Cost', 'Profit'],
  'attributes': ['Category', 'Call Center']},
 'filter': {'Year': {'In': [2021, 2022]},
  'Call Center': {'Equals': 'Atlanta'},
  'Revenue': {'Greater': 200}}}

In [ ]:
#filter_d=filter_d['criteria']


### Format challange noch offen

In [ ]:
filter_d = {"year": {"In": [2021, 2022]}, "revenue": {"Greater": 200}, "region": {"BeginsWith": "S"}}
#filter_d = {"year": {"In": [2021, 2022]} and "revenue": {"Percent.Top": 10,"level":["Customers"]} and "region": {"BeginsWith": "S"}}

## interpret AI report definition. Filter only

In [8]:
ai_prp_filt_ans_d=i_run_chat_request.bld_AI_filt_prp_ans(conn,vector_store=i_keyword_processor,filter_d=filter_d)
merged_filt_prp_l=i_run_chat_request.merge_exp_prp_ans_l(ai_prp_filt_ans_d)
merged_filt_prp_l

Year
Year
OlapCube object named: 'obj_prp_rel' with ID: '1F0C564A40101C08ABF602AAB81B2324'


OlapCube object named: 'attribute_form_elements' with ID: 'B0A4AFCB4F8B53B4EB677D89B7A85CF0'


Call Center
Call Center
OlapCube object named: 'obj_prp_rel' with ID: '1F0C564A40101C08ABF602AAB81B2324'


OlapCube object named: 'att_form_def' with ID: '870C97DA4303C51097C4068CE4DB6C4D'


Revenue
Revenue
OlapCube object named: 'obj_prp_rel' with ID: '1F0C564A40101C08ABF602AAB81B2324'


[{'id': '1F21042E4FC08886A9D7E4920CA4EC59',
  'type': 'EXPRESSION',
  'answers': {'expression': {'operator': 'And',
    'operands': [{'operator': 'AND',
      'operands': [{'operator': 'In',
        'operands': [{'type': 'form',
          'attribute': {'id': '8D679D5111D3E4981000E787EC6DE8A4'},
          'form': {'id': '45C11FA478E745FEA08D781CEA190FE5'}},
         {'type': 'constants',
          'dataType': 'Numeric',
          'values': ['2021', '2022']}]},
       {'operator': 'Equals',
        'operands': [{'type': 'form',
          'attribute': {'id': '8D679D3511D3E4981000E787EC6DE8A4'},
          'form': {'id': 'CCFBE2A5EADB4F50941FB879CCF1721C'}},
         {'type': 'constant', 'dataType': 'Char', 'value': 'Atlanta'}]},
       {'operator': 'Equals',
        'operands': [{'type': 'form',
          'attribute': {'id': '8D679D3511D3E4981000E787EC6DE8A4'},
          'form': {'id': '035E890A4157E9828414E8A0E13E2998'}},
         {'type': 'constant', 'dataType': 'Char', 'value': 'Atlanta

## parse object prompts for temple definition

In [9]:
obj_prp_ans_l=i_run_chat_request.bld_ai_obj_ans_prp(conn,json_t_d )
obj_prp_ans_l  

OlapCube object named: 'obj_prp_rel' with ID: '1F0C564A40101C08ABF602AAB81B2324'


[{'id': '2AF7E8744FE8F3FEA97110AC3A3AA5EF',
  'type': 'OBJECTS',
  'answers': [{'id': '8D679D3511D3E4981000E787EC6DE8A4', 'type': 'attribute'},
   {'id': '8D679D3711D3E4981000E787EC6DE8A4', 'type': 'attribute'}]},
 {'id': '6BB3C88A44CF937F0770B4BE22E86118',
  'type': 'OBJECTS',
  'answers': [{'id': '4C05177011D3E877C000B3B2D86C964F', 'type': 'metric'},
   {'id': '4C051DB611D3E877C000B3B2D86C964F', 'type': 'metric'}]}]

In [10]:
merged_filt_prp_l.extend(obj_prp_ans_l)

prompt_answ=i_prompts.frame_prp(prp_ans=merged_filt_prp_l) 
prompt_answ

'{"prompts": [{"id": "1F21042E4FC08886A9D7E4920CA4EC59", "type": "EXPRESSION", "answers": {"expression": {"operator": "And", "operands": [{"operator": "AND", "operands": [{"operator": "In", "operands": [{"type": "form", "attribute": {"id": "8D679D5111D3E4981000E787EC6DE8A4"}, "form": {"id": "45C11FA478E745FEA08D781CEA190FE5"}}, {"type": "constants", "dataType": "Numeric", "values": ["2021", "2022"]}]}, {"operator": "Equals", "operands": [{"type": "form", "attribute": {"id": "8D679D3511D3E4981000E787EC6DE8A4"}, "form": {"id": "CCFBE2A5EADB4F50941FB879CCF1721C"}}, {"type": "constant", "dataType": "Char", "value": "Atlanta"}]}, {"operator": "Equals", "operands": [{"type": "form", "attribute": {"id": "8D679D3511D3E4981000E787EC6DE8A4"}, "form": {"id": "035E890A4157E9828414E8A0E13E2998"}}, {"type": "constant", "dataType": "Char", "value": "Atlanta"}]}]}]}}}, {"id": "14B57F1F46127BC6E6F746888BC3C7CB", "type": "EXPRESSION", "answers": {"expression": {"operator": "AND", "operands": [{"operator

In [11]:
ai_rep_name="dyn_prompt_page_botstat"
report_id="25D40AD444B6D51B333021ADFB219501"
#report_id="29FBB4884282ED7203AFD2B247C57688"

rep_id=i_run_chat_request.save_AI_rep(conn=conn,report_id=report_id
                                  ,prompt_answ=prompt_answ
                                  ,ai_rep_name=ai_rep_name
                                  ,ai_rep_folder_id=ai_rep_folder_id)
new_rep_id=rep_id.json()["id"]
instance_id = i_rep.open_Instance(conn=conn, report_id=new_rep_id)
df=i_rep.report_df(conn=conn, report_id=new_rep_id, instance_id=instance_id)
df

Report object named: 'dyn_prompt_page_botstat' with ID: 'C731B12148180FDA527FB5BCE95E618E'


,Category,Call Center@DESC,Call Center@Hyperlink,Profit,Revenue


In [12]:
ai_rep_name="dyn_prompt_page_botstat"
report_id="25D40AD444B6D51B333021ADFB219501"
#report_id="29FBB4884282ED7203AFD2B247C57688"

rep_id=i_run_chat_request.save_AI_rep(conn=conn,report_id=report_id
                                  ,prompt_answ=prompt_answ
                                  ,ai_rep_name=ai_rep_name
                                  ,promptOption ="filterAndTemplate"
                                  ,ai_rep_folder_id=ai_rep_folder_id)
new_rep_id=rep_id.json()["id"]
link=i_rep.web_base_url(conn=conn,report_id=new_rep_id)
HTML(link)

http://85.214.60.83:8080/MicroStrategy/servlet/mstrWeb?Server=85.214.60.83&Project=MicroStrategy+Tutorial&evt=4001&src=mstrWeb.4001&reportViewMode=1&reportID=C731B12148180FDA527FB5BCE95E618E&currentViewMedia=2


In [ ]:
link=i_rep.bld_rep_library_url(conn=conn,report_id=new_rep_id)
HTML(link)

In [ ]:
print(i_rep.get_open_prp_stat(conn=conn, report_id=report_id, instance_id=instance_id))


In [ ]:
from mstrio.api import reports

report_ds

In [ ]:
report_ds.json()

In [ ]:
conn.base_url

In [ ]:
dir(conn)

In [ ]:

extract_server_port(base_url=conn.base_url)

In [ ]:
url_parts = conn.base_url.split("/") 
server=url_parts[2].split(":")
server[0]

In [ ]:
conn.project_name.replace(" ","+")